In [ ]:
!pip install transformers datasets accelerate peft


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
raw_ds = load_dataset("cnn_dailymail", "3.0.0")
train_ds = raw_ds["train"].shuffle(seed=42).select(range(5000))
val_ds   = raw_ds["validation"].shuffle(seed=42).select(range(500))


In [ ]:
model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


In [ ]:
peft_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05
)
model = get_peft_model(model, peft_config)


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

def preprocess(batch):
    inputs = tokenizer(batch["article"], truncation=True, padding="max_length", max_length=512)
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(batch["highlights"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_tok = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_tok   = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./summarizer_lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=1,
    predict_with_generate=True,
    fp16=True,
    logging_steps=100,
    # evaluation_strategy="steps",
    eval_steps=200,
    save_steps=500,
    num_train_epochs=3,
    learning_rate=3e-4,
    weight_decay=0.01,
    save_total_limit=2,

    report_to="none",
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()


In [ ]:
model.save_pretrained("./summarizer_lora_final")

In [ ]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="./summarizer_lora_final",
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)


In [ ]:

article = """
The Ministry of Health announced today the launch of a nationwide vaccination campaign aiming to immunize over 20 million children against measles and rubella within the next six months. This ambitious effort is part of the government’s broader public health initiative to eradicate preventable infectious diseases and strengthen primary healthcare services across all governorates.

According to Dr. Sarah Mitchell, Director of the National Immunization Program, the campaign will deploy mobile clinics to reach remote and underserved communities, ensuring that children in rural areas have equal access to vaccines. The program also includes training sessions for over 5,000 healthcare workers on cold-chain management and community engagement strategies to address vaccine hesitancy.

Financial support for the campaign comes from a partnership between the Ministry of Health, UNICEF, and the World Health Organization, with an estimated budget of USD 85 million. UNICEF’s regional director, Mr. Ahmed Saleh, emphasized the importance of collaboration between international agencies and local authorities to ensure the smooth distribution of vaccine supplies and the monitoring of adverse events following immunization.

Pilot operations have already begun in three governorates, with preliminary reports indicating high turnout rates and positive community feedback. The Ministry plans to roll out the full campaign next week, launching an extensive media outreach program that includes radio spots, social media engagement, and school-based informational seminars.

Officials predict that by achieving at least a 95% coverage rate, the campaign will help the country maintain its polio-free status and pave the way for future public health achievements, including the integration of routine childhood vaccines into digital health records. The success of this campaign is seen as a critical step toward meeting the Sustainable Development Goal of ensuring healthy lives for all by 2030.
"""

print("=== Summary: ===")
print(
    summarizer(
        article,
        max_length=150,
        min_length=40,
        do_sample=False
    )[0]["summary_text"]
)

In [ ]:


from transformers import pipeline

summarizer_cpu = pipeline(
    "summarization",
    model="./summarizer_lora_final",
    tokenizer=tokenizer,
    device= -1
)


# === Very Large Article ===
article = """
Artificial Intelligence (AI) has rapidly evolved from a futuristic concept in science fiction to a pivotal technology transforming industries, societies, and personal lives. The 21st century has witnessed unprecedented developments in AI capabilities, from machine learning and natural language processing to robotics and computer vision. This article delves into the evolution of AI, its current applications, challenges, and its projected impact on the future.

**Historical Context and Evolution**

The term "Artificial Intelligence" was first coined in 1956 by John McCarthy at the Dartmouth Conference, where researchers gathered to explore ways to make machines simulate human intelligence. Early AI systems focused on rule-based logic, which limited their capabilities. It wasn't until the rise of computational power and access to large datasets that AI started making significant strides.

In the 1980s and 1990s, expert systems attempted to mimic decision-making of human specialists, but they lacked adaptability. The 2000s marked a turning point with the resurgence of neural networks and the birth of deep learning. Algorithms could now "learn" from data rather than being explicitly programmed.

**Key Breakthroughs**

Several key breakthroughs accelerated AI’s development. The introduction of convolutional neural networks (CNNs) allowed AI systems to excel in image recognition tasks. Recurrent neural networks (RNNs) and later transformers revolutionized natural language processing, leading to models capable of real-time language translation, summarization, and conversation.

In 2012, the ImageNet competition was won by a deep learning model that vastly outperformed others in image classification, cementing the utility of deep learning. By 2018, OpenAI’s GPT series and Google’s BERT demonstrated the power of large language models, capable of human-like text generation.

**Applications Across Industries**

Today, AI applications span almost every industry:

- **Healthcare**: AI aids in disease diagnosis, drug discovery, and patient monitoring. Algorithms analyze medical images, predict patient deterioration, and assist in robotic surgeries.
- **Finance**: AI powers fraud detection, algorithmic trading, and customer service chatbots. Machine learning models assess credit risk and automate compliance.
- **Retail**: From personalized recommendations to supply chain optimization, AI transforms customer experience and inventory management.
- **Transportation**: Autonomous vehicles, route optimization, and predictive maintenance are driven by AI algorithms.
- **Education**: Adaptive learning platforms personalize education, while AI grading systems reduce administrative load on educators.
- **Entertainment**: AI curates personalized content on streaming platforms, generates music, and even writes screenplays.

**AI in Daily Life**

Beyond industry, AI pervades everyday life. Voice assistants like Siri and Alexa understand and respond to voice commands. Social media platforms use AI to filter content, detect harmful posts, and serve personalized ads. Smartphones use AI in cameras, keyboards, and battery management.

**Ethical Concerns and Challenges**

Despite its promise, AI raises several concerns:

- **Bias and Fairness**: AI models trained on biased data can perpetuate and amplify discrimination in hiring, lending, and law enforcement.
- **Privacy**: Surveillance systems and data-driven models often infringe on individual privacy rights.
- **Job Displacement**: Automation threatens to displace workers in manufacturing, customer service, and even white-collar roles.
- **Autonomy and Accountability**: As AI systems make more autonomous decisions, questions arise over who is responsible for errors or harm caused by these systems.

Efforts are underway to make AI more transparent, interpretable, and accountable. Governments and organizations are developing ethical frameworks and regulatory policies to ensure responsible AI development.

**The Future of AI**

AI is expected to further integrate into human life, with advances in general intelligence, where machines might learn and reason across multiple domains like humans. Quantum computing may exponentially increase AI capabilities, enabling real-time simulations of complex systems in healthcare, climate, and physics.

Emerging fields like neuro-symbolic AI aim to combine the learning ability of neural networks with logical reasoning, bringing us closer to explainable AI. Human-AI collaboration will redefine workplaces, education, and creativity.

**Conclusion**

Artificial Intelligence is no longer a distant dream but a present reality shaping every aspect of modern life. Its potential is vast, but so are its challenges. Navigating the future of AI requires a balance between innovation and ethics, automation and employment, efficiency and equity. As we stand at the cusp of a new era, the way we guide AI development will determine whether it becomes humanity’s greatest tool or its most formidable challenge.
"""

article2 = """
Artificial Intelligence (AI) has rapidly evolved from a futuristic concept in science fiction to a pivotal technology transforming industries, societies, and personal lives. The 21st century has witnessed unprecedented developments in AI capabilities, from machine learning and natural language processing to robotics and computer vision. This article delves into the evolution of AI, its current applications, challenges, and its projected impact on the future.

**Historical Context and Evolution**

The term "Artificial Intelligence" was first coined in 1956 by John McCarthy at the Dartmouth Conference, where researchers gathered to explore ways to make machines simulate human intelligence. Early AI systems focused on rule-based logic, which limited their capabilities. It wasn't until the rise of computational power and access to large datasets that AI started making significant strides.

In the 1980s and 1990s, expert systems attempted to mimic decision-making of human specialists, but they lacked adaptability. The 2000s marked a turning point with the resurgence of neural networks and the birth of deep learning. Algorithms could now "learn" from data rather than being explicitly programmed.

**Key Breakthroughs**

Several key breakthroughs accelerated AI’s development. The introduction of convolutional neural networks (CNNs) allowed AI systems to excel in image recognition tasks. Recurrent neural networks (RNNs) and later transformers revolutionized natural language processing, leading to models capable of real-time language translation, summarization, and conversation.

In 2012, the ImageNet competition was won by a deep learning model that vastly outperformed others in image classification, cementing the utility of deep learning. By 2018, OpenAI’s GPT series and Google’s BERT demonstrated the power of large language models, capable of human-like text generation.

**Applications Across Industries**

Today, AI applications span almost every industry:

- **Healthcare**: AI aids in disease diagnosis, drug discovery, and patient monitoring. Algorithms analyze medical images, predict patient deterioration, and assist in robotic surgeries.
"""

# === Zero-shot Prompt ===
zero_shot_prompt = (
    "Summarize the following article into key points:\n\n"
    + article
    + "\n\nSummary:"
)
example = (
    "Example:\n"
    "Article: Artificial Intelligence has rapidly evolved in the 21st century. It is now used in many industries including healthcare and finance.\n"
    "Summary: AI has evolved quickly and is now used across multiple industries.\n\n"
)

# === Few-shot Prompt ===
few_shot_prompt = (
    example +
    "Now summarize the following article into key points:\n\n" +
    article2 +
    "\n\nSummary:"
)

def clean_prompt(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

# zero_shot_prompt = clean_prompt(zero_shot_prompt)[:3500]
# few_shot_prompt = clean_prompt(few_shot_prompt)[:3500]

# === Generate summaries ===
print("=== Zero-shot Summary ===")
zero_result = summarizer_cpu(zero_shot_prompt, max_length=150, min_length=40, do_sample=False)
print(zero_result[0]["summary_text"])

print("\n=== Few-shot Summary ===")
few_result = summarizer_cpu(few_shot_prompt, max_length=150, min_length=40, do_sample=False)
print(few_result[0]["summary_text"]
)

